# ConversationKGMemory
- 지식 그래프를 활용하여 정보를 저장하고 불러온다.
- 이를 통해 LLM 모델이 서로 다른 개체 간의 관계를 이해하는데 도움을 주고, 복잡한 연결망과 역사적 맥락을 기반으로 대응하는 능력을 향상시킨다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


In [2]:
from langchain.memory import ConversationKGMemory

In [11]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models import ChatOllama

# llm = HuggingFaceEndpoint(
#     repo_id=os.environ["MODEL_ID"], 
#     max_new_tokens=1024,
#     temperature=0.1,
#     huggingfacehub_api_token=os.environ["HF_API_KEY"],
# )
# model = ChatHuggingFace(llm=llm)
model = ChatOllama(model="mistral:7b")
# model = ChatOllama(model="gemma:7b")

In [12]:
memory = ConversationKGMemory(llm=model, return_messages=True)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})


In [13]:
memory.load_memory_variables({"input": "who is sam"})

{'history': []}

In [14]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=model, prompt=prompt, memory=ConversationKGMemory(llm=model)
)

In [15]:
conversation_with_kg.invoke(input="My name is Teddy. I'm Software Engineer. Shirley is a coworker of mine, and she's a new designer at our company.")

{'input': "My name is Teddy. I'm Software Engineer. Shirley is a coworker of mine, and she's a new designer at our company.",
 'history': '',
 'response': " Hello Teddy, it's nice to meet you! As a software engineer at your company, you work with various teams, including the design team where Shirley, the new designer, belongs. I don't have specific details about Shirley or her previous experience, but I can assist you in understanding design concepts and technologies if needed for your collaboration with her.\n\nI can also provide information about design principles such as user-centered design, color theory, typography, and layout techniques. Additionally, I can help answer questions related to software development, programming languages, databases, or other technical topics that might arise during your work together. Let me know if you have any specific inquiries!"}

In [16]:
conversation_with_kg.memory.load_memory_variables({"input": "who is Shirley?"})

{'history': 'On Shirley: Shirley is a designer. Shirley works at our company.'}

In [17]:
conversation_with_kg.invoke(input="Who is Shirley?")

{'input': 'Who is Shirley?',
 'history': 'On Shirley: Shirley is a designer. Shirley works at our company.',
 'response': ' Shirley is a designer who works at our company.'}

In [18]:
conversation_with_kg.invoke(input="Who is my coworker?")

{'input': 'Who is my coworker?',
 'history': '',
 'response': " I don't have any information about your coworkers, as we haven't covered that topic in our context yet. However, if you provide me with more details such as the name of the company or project you are working on, I might be able to help find information related to your coworkers."}